# REST API BACKEND

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List
import sys

# Ensure correct Python environment
if "micropip" in sys.modules:
    del sys.modules["micropip"]

app = FastAPI()

# Request Model
class RequestModel(BaseModel):
    data: List[str]

# POST Endpoint
@app.post("/bfhl")
def process_data(request: RequestModel):
    user_id = "john_doe_17091999"  # Hardcoded example user_id
    email = "john@xyz.com"
    roll_number = "ABCD123"
    
    numbers = [x for x in request.data if x.isdigit()]
    alphabets = [x for x in request.data if x.isalpha()]
    highest_alphabet = [max(alphabets, key=str.upper)] if alphabets else []
    
    response = {
        "is_success": True,
        "user_id": user_id,
        "email": email,
        "roll_number": roll_number,
        "numbers": numbers,
        "alphabets": alphabets,
        "highest_alphabet": highest_alphabet
    }
    return response

# GET Endpoint
@app.get("/bfhl")
def get_operation_code():
    return {"operation_code": 1}

# For local testing
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)


# Frontend NODEJS

In [ ]:
import { useState } from "react";
import Head from "next/head";
import axios from "axios";

export default function Home() {
  const [jsonInput, setJsonInput] = useState("");
  const [responseData, setResponseData] = useState({
    alphabets: [],
    numbers: [],
    highest_alphabet: []
  });
  const [selectedOptions, setSelectedOptions] = useState([]);
  const [error, setError] = useState(null);

  const handleSubmit = async () => {
    try {
      if (!jsonInput.trim()) {
        throw new Error("Input cannot be empty");
      }
      
      const parsedData = JSON.parse(jsonInput);
      if (!parsedData.data || !Array.isArray(parsedData.data)) {
        throw new Error("Invalid JSON format. Expected { data: [] }");
      }
      
      setError(null);
      const response = await axios.post("http://localhost:8000/bfhl", parsedData);
      setResponseData({
        alphabets: response.data.alphabets || [],
        numbers: response.data.numbers || [],
        highest_alphabet: response.data.highest_alphabet || []
      });
    } catch (err) {
      setError(err.message);
      setResponseData({
        alphabets: [],
        numbers: [],
        highest_alphabet: []
      });
    }
  };

  const renderResponse = () => {
    if (!responseData || Object.keys(responseData).length === 0) return null;
    let displayData = {};
    if (selectedOptions.includes("Alphabets")) displayData.alphabets = responseData.alphabets;
    if (selectedOptions.includes("Numbers")) displayData.numbers = responseData.numbers;
    if (selectedOptions.includes("Highest Alphabet")) displayData.highest_alphabet = responseData.highest_alphabet;
    return <pre>{JSON.stringify(displayData, null, 2)}</pre>;
  };

  return (
    <>
      <Head>
        <title>ABCD123</title>
      </Head>
      <div style={{ padding: 20 }}>
        <h1>JSON Input</h1>
        <textarea
          rows={5}
          cols={50}
          value={jsonInput}
          onChange={(e) => setJsonInput(e.target.value)}
          placeholder='{"data": ["A", "C", "Z"]}'
        ></textarea>
        <br />
        <button onClick={handleSubmit}>Submit</button>
        {error && <p style={{ color: "red" }}>{error}</p>}
        {responseData && Object.values(responseData).some(arr => Array.isArray(arr) && arr.length > 0) && (
          <>
            <h2>Select Data to Display</h2>
            <select multiple value={selectedOptions} onChange={(e) => setSelectedOptions([...e.target.selectedOptions].map(o => o.value))}>
              <option value="Alphabets">Alphabets</option>
              <option value="Numbers">Numbers</option>
              <option value="Highest Alphabet">Highest Alphabet</option>
            </select>
            {renderResponse()}
          </>
        )}
      </div>
    </>
  );
}
